In [1]:
import pandas as pd
import plotly.express as px

# 1. Carregar dataset
df = pd.read_csv('static/Billionaires Statistics Dataset.csv')

# 2. Configurar colunas
country_col = 'country'
industry_col = 'industries'
net_col = 'finalWorth'

# 3. Criar coluna de Região (exemplo de mapeamento manual)
continent_map = {
    'United States': 'North America',
    'China': 'Asia',
    'India': 'Asia',
    'Germany': 'Europe',
    'France': 'Europe',
    'United Kingdom': 'Europe',
    'Russia': 'Europe',
    'Brazil': 'South America',
    'Canada': 'North America',
    'Australia': 'Oceania'
}
df['Region'] = df[country_col].map(continent_map).fillna('Other')

# 4. Agregar patrimônio líquido por Região → País → Indústria
agg = (
    df.groupby(['Region', country_col, industry_col])[net_col]
      .sum()
      .reset_index()
      .rename(columns={country_col: 'Country', industry_col: 'Industry', net_col: 'NetWorth'})
)

# 5. Gerar gráfico Sunburst hierárquico
fig = px.sunburst(
    agg,
    path=['Region', 'Country', 'Industry'],
    values='NetWorth',
    title='Distribuição Hierárquica do Patrimônio Líquido de Bilionários'
)
fig.show()


In [4]:
import pandas as pd
import plotly.graph_objects as go


# Seleciona colunas e agregações:
# Top 5 países por soma de patrimônio e top 5 indústrias
top_countries = df.groupby('country')['finalWorth'].sum().nlargest(5).index.tolist()
top_industries = df.groupby('industries')['finalWorth'].sum().nlargest(5).index.tolist()

# Agrupa fluxos entre esses países e indústrias
flow = (
    df[df['country'].isin(top_countries) & df['industries'].isin(top_industries)]
    .groupby(['country', 'industries'])['finalWorth']
    .sum()
    .reset_index(name='totalWorth')
)

# Prepara labels únicos
labels = list(pd.unique(flow[['country', 'industries']].values.ravel()))

# Mapeia fontes e destinos para índices
source_indices = flow['country'].apply(lambda x: labels.index(x))
target_indices = flow['industries'].apply(lambda x: labels.index(x))
values = flow['totalWorth']

# Cria o diagrama Sankey
fig = go.Figure(go.Sankey(
    arrangement="snap",
    node=dict(label=labels, pad=15, thickness=20),
    link=dict(source=source_indices, target=target_indices, value=values)
))

fig.update_layout(
    title_text="Fluxo de Riqueza (finalWorth) por País → Indústria (Top 5 cada)",
    font_size=10
)


In [2]:
df.columns


Index(['rank', 'finalWorth', 'category', 'personName', 'age', 'country',
       'city', 'source', 'industries', 'countryOfCitizenship', 'organization',
       'selfMade', 'status', 'gender', 'birthDate', 'lastName', 'firstName',
       'title', 'date', 'state', 'residenceStateRegion', 'birthYear',
       'birthMonth', 'birthDay', 'cpi_country', 'cpi_change_country',
       'gdp_country', 'gross_tertiary_education_enrollment',
       'gross_primary_education_enrollment_country', 'life_expectancy_country',
       'tax_revenue_country_country', 'total_tax_rate_country',
       'population_country', 'latitude_country', 'longitude_country',
       'Region'],
      dtype='object')